<a href="https://colab.research.google.com/github/munibaweb123/Q4-assignment-01/blob/main/litellm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents  "openai-agents[litellm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00


# **Make your Jupyter Notebook capable of running asynchronous functions.**

In [2]:
import nest_asyncio
nest_asyncio.apply()

# **Run Google Gemini with LiteLLm and OPENAI-Agent SDK**
## Function Run Sync

In [3]:
import asyncio
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from google.colab import userdata

set_tracing_disabled(disabled=True)

MODEL = 'gemini/gemini-2.0-flash'
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

def main(model:str, api_key:str):
  agent = Agent(
      name = "assistant",
      instructions = "You only respond in haikus",
      model = LitellmModel(model=model, api_key=api_key)
  )
  result = Runner.run_sync(agent, "what is an AI agent?")
  print(result.final_output)

main(model=MODEL, api_key=GEMINI_API_KEY)


Code that thinks and acts,
Reaching goals on its own,
Smart machine awakes.



/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='Code tha...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
